In [5]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect,requests
from selenium import webdriver# Controls the browser
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
import pandas as pd
import numpy as np
from io import StringIO

amfii_path = r"https://www.amfiindia.com/otherdata/fund-performance"

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get(amfii_path)
wait = WebDriverWait(driver, 10)

# Switch to iframe
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

dropdown_1 = ['Open Ended', 'Close Ended']
dropdown_2 = ['Equity', 'Debt', 'Hybrid', 'Solution Oriented', 'Other']
fixed_op4 = "All"

def select_dropdown(dropdown_element, value):
    arrow = dropdown_element.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
    arrow.click()
    time.sleep(0.5)
    options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
    for opt in options:
        if opt.text.strip() == value:
            opt.click()
            break
    time.sleep(0.5)

for op1 in dropdown_1:
    dropdowns = driver.find_elements(By.CSS_SELECTOR, ".ng-select")
    select_dropdown(dropdowns[0], op1)

    for op2 in dropdown_2:
        dropdowns = driver.find_elements(By.CSS_SELECTOR, ".ng-select")
        select_dropdown(dropdowns[1], op2)

        # Get all op3 options
        dropdowns = driver.find_elements(By.CSS_SELECTOR, ".ng-select")
        arrow = dropdowns[2].find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
        arrow.click()
        time.sleep(0.5)
        options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
        op3_options = [opt.text.strip() for opt in options]
        arrow.click()

        for op3 in op3_options:
            dropdowns = driver.find_elements(By.CSS_SELECTOR, ".ng-select")
            select_dropdown(dropdowns[2], op3)
            select_dropdown(dropdowns[3], fixed_op4)

            # Click "Go" button
            go_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.mat-raised-button")))
            go_button.click()
            # print(f"✅ Selected: {op1} + {op2} + {op3} + {fixed_op4}")
            # time.sleep(5) 
            # Set the date to 30-Sep-2025
            date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[formcontrolname='AsOnDate']")))
            date_input.clear()
            date_input.send_keys("30-Sep-2025")
            time.sleep(1)

            # Click the "Download Excel" button
            download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Download Excel')]")))
            # download_button.click()
            driver.execute_script("arguments[0].click();", download_button)
            print(f"📥 Download triggered for: {op1} + {op2} + {op3} + {fixed_op4}")
            time.sleep(1)  # wait for download to complete

driver.quit()

In [2]:
dropdown_1 = ['Open Ended', 'Close Ended']
op3_map = {
    "Equity": ['Large Cap', 'Large & Mid Cap', 'Flexi Cap', 'Multi Cap', 'Mid Cap',
               'Small Cap', 'Value', 'ELSS', 'Contra', 'Dividend Yield', 'Focused', 'Sectoral / Thematic'],
    "Debt": ['Long Duration', 'Medium to Long Duration', 'Short Duration', 'Medium Duration',
             'Money Market', 'Low Duration', 'Ultra Short Duration', 'Liquid', 'Overnight',
             'Dynamic Bond', 'Corporate Bond', 'Credit Risk', 'Banking and PSU', 'Floater',
             'FMP', 'Gilt', 'Gilt with 10 year constant duration'],
    "Hybrid": ['Aggressive Hybrid', 'Conservative Hyrbid', 'Equity Savings', 'Arbitrage',
               'Multi Asset Allocation', 'Dynamic Asset Allocation or Balanced Advantage', 'Balanced Hybrid'],
    "Solution Oriented": ["Children's", 'Retirement'],
    "Other": ['Index Funds ETFs', 'FoFs (Overseas/Domestic)']
}

dropdown_4_value = "All"

all_combos = []
for op1 in dropdown_1:
    for op2, op3_list in op3_map.items():
        for op3 in op3_list:
            all_combos.append({
                "op1": op1,
                "op2": op2,
                "op3": op3,
                "op4": dropdown_4_value
            })

# print(f"Total combinations: {len(all_combos)}")
# all_combos
# Small test set
# combos = [
#     {"op1": "Open Ended", "op2": "Debt", "op3": "Overnight", "op4": "All"},
#     {"op1": "Open Ended", "op2": "Debt", "op3": "Liquid", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Large Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Large & Mid Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Flexi Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Multi Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Mid Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Small Cap", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Value", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "ELSS", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Contra", "op4": "All"},
    # {"op1": "Open Ended", "op2": "Equity", "op3": "Dividend Yield", "op4": "All"},
# ]

In [ ]:
import os, time, json, shutil, glob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import pandas as pd

DATE_STR = "30-Jul-2025"
DOWNLOAD_DIR = os.path.join(os.getcwd(), f"mf_downloads_{DATE_STR}")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

driver.get("https://www.amfiindia.com/otherdata/fund-performance")
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

combos = all_combos

def select_dropdown(index, value, retries=3):
    for attempt in range(retries):
        try:
            # Wait for overlay to disappear if present
            try:
                WebDriverWait(driver, 5).until(
                    EC.invisibility_of_element_located((By.CSS_SELECTOR, ".cdk-overlay-backdrop"))
                )
            except TimeoutException:
                pass  # Continue anyway

            dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
            dropdown = dropdowns[index]
            driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)

            arrow = dropdown.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
            driver.execute_script("arguments[0].click();", arrow)
            time.sleep(0.5)

            options = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option"))
            )

            for opt in options:
                if opt.text.strip() == value:
                    driver.execute_script("arguments[0].scrollIntoView(true);", opt)
                    driver.execute_script("arguments[0].click();", opt)
                    time.sleep(0.5)
                    return

            raise ValueError(f"Option '{value}' not found")
        except Exception as e:
            print(f"Retrying select for '{value}' ({attempt+1}/{retries}) due to {e.__class__.__name__}")
            time.sleep(1)
    raise Exception(f"Failed to select '{value}' after retries")

# def select_dropdown(index, value, retries=3):
#     for attempt in range(retries):
#         try:
#             dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
#             dropdown = dropdowns[index]
#             arrow = dropdown.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
#             arrow.click()
#             time.sleep(0.5)

#             options = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
#             for opt in options:
#                 if opt.text.strip() == value:
#                     driver.execute_script("arguments[0].scrollIntoView(true);", opt)
#                     # opt.click()
#                     driver.execute_script("arguments[0].click();", opt)
#                     time.sleep(0.5)
#                     return
#             raise ValueError(f"Option '{value}' not found")
#         except Exception as e:
#             print(f"Retrying select for '{value}' ({attempt+1}/{retries}) due to {e.__class__.__name__}")
#             time.sleep(1)
#     raise Exception(f"Failed to select '{value}' after retries")

def set_date(date_str):
    date_input = driver.find_element(By.CSS_SELECTOR, "input[bsdatepicker]")
    driver.execute_script("""
        arguments[0].value = arguments[1];
        arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
        arguments[0].dispatchEvent(new Event('change', { bubbles: true }));
    """, date_input, date_str)
    time.sleep(1)

def click_go():
    go_button = driver.find_element(By.CSS_SELECTOR, "button[mat-raised-button]")
    go_button.click()
    time.sleep(2)

def download_excel():
    excel_icon = driver.find_element(By.CSS_SELECTOR, ".excel-border.cursor-pointer")
    excel_icon.click()
    time.sleep(1)

def if_data_avalaible():
    try:
        snackbars = driver.find_elements(By.CSS_SELECTOR, "snack-bar-container")
        for snackbar in snackbars:
            if "Data Not Available" in snackbar.text:
                close_icon = snackbar.find_element(By.XPATH, ".//mat-icon[contains(text(), 'close')]")
                close_icon.click()
                time.sleep(1)
                print("Data Not Available. Snackbar closed.")
                return False
    except:
        pass
    return True


for combo in combos:
    try:
        op1, op2, op3, op4 = combo["op1"], combo["op2"], combo["op3"], combo["op4"]

        print(f"\nPreparing combo: {op1}, {op2}, {op3}, {op4}")
    
        set_date(DATE_STR)
        select_dropdown(0, op1)
        select_dropdown(1, op2)
        select_dropdown(2, op3)
        select_dropdown(3, op4)

        # click_go()
        time.sleep(2)

    except Exception as e:
        print(f"[ERROR] {combo}: {e}")

driver.quit()


In [ ]:
df = pd.read_excel("FundPerformance_30Sep2025.xlsx")
df.head()

In [18]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9


In [ ]:
def create_grand_sheet(path:str):
    all_dataframes = []
    excel_files = sorted(glob.glob(os.path.join(path, "*.xlsx")))
    
    all_data_path = os.path.join(path, "ALL_DATA_FILE.xlsx")
    grand_path = os.path.join(path, f"Grand_Stacked_{DATE_STR}.xlsx")
    
    #stack
    for file in excel_files:
        df = pd.read_excel(file, header=None)
        empty_rows = pd.DataFrame([[""] * df.shape[1]] * 3)
        all_dataframes.append(df)
        all_dataframes.append(empty_rows)

    grand_df = pd.concat(all_dataframes, ignore_index=True)
    grand_df.to_excel(grand_path, index=False, header=False)
    
    #one shot
    with pd.ExcelWriter(all_data_path, engine='openpyxl') as writer:
        for i, file in enumerate(excel_files):
            sheet_name = f"Sheet{i+1}"
            df = pd.read_excel(file, header=None)
            df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

    print(f" `ALL_DATA_FILE` created with {len(excel_files)} sheets: {all_data_path}")
    print(f"Grand Excel created with {len(excel_files)} : {grand_path}")


path = r"C:\Users\kaustubh.keny\Projects\office-work\selenium-scrape\mf_downloads_30-Jul-2025\mod"

create_grand_sheet(path)
